## Competicion Algoritmos MIA-X ##
### Ejemplo de preparación de algoritmos ###

In [ ]:
import pandas as pd
import requests, json

La comunicación via API se realiza a través de la siguiente dirección web

In [ ]:
url_base = 'https://miax-gateway-jog4ew3z3q-ew.a.run.app'
headers = {'Content-Type': 'application/json'}

Es necesario indicar en la petición el identificador 
de la competición y la clave personal

In [ ]:
competi = 'mia_8'
user_key = 'PON_AQUI_TU_KEY'

### Notas sobre los métodos HTTP ###
En la comunicación via API se utiliza el protocolo HTTP que entre otros
permite los métodos GET y POST.  Hay que tomar especial antención a la 
construcción de la petición. En ambos casos la key de autenticación
va en el URL.
- GET: los parámetros se añaden junto con la key en la URL
- POST: los parámetros van en el cuerpo del mensaje

### Consulta algoritmos del usuario ###
Cada participante se crea automáticamente con tres algoritmos tipo allocator.
Cada algoritmo tiene asociada una etiqueta o 'algo_tag' que se utiliza como identificador a la hora de hacer operaciones
Cada algoritmo puede tener allocations para cada uno de los indices haciendo un total de 9 algoritmos.

In [ ]:
url = f'{url_base}/participants/algorithms'
params = {'competi': competi,
          'key': user_key}
response = requests.get(url, params)
algos = response.json()
if algos:
    algos_df = pd.DataFrame(algos)
    print(algos_df.to_string())


In [ ]:
algo_tag = algos_df.iloc[1].algo_tag
algo_tag

### Agregar una asignacion de pesos a un algoritmo ###
Los algoritmos tipo allocator corresponden a la gestión de los pesos de una
cartera de inversión.  Una asignación de capital o 'allocation' es la proporción
de la cartera que se desea para un día concreto.  El backtesing se encargará
de transformar esas asignaciones en órdenes de compra/venta cuando la cartera
actual difiera de la asignación deseada. **Solo es necesario indicar un allocation
cuando se desee realizar una re-asignación**.  El allocation nunca debe sumar más 
de uno.  No incluir un ticker se infiere como peso 0.

In [ ]:
def gen_alloc_data(ticker, alloc):
    return {'ticker': ticker,
            'alloc': alloc}

In [ ]:
tickers = ['TEF','SAN','IBE','ITX','REP','AMS']

In [ ]:
market = 'IBEX'

Formato de una asignación. En este ejemplo generaremos 
la misma asignación una vez al mes durante 3 meses para
que la cartera quede rebalanceada cada vez, guardando 
un 10% de efectivo

In [ ]:
allocation = [gen_alloc_data(tk, 0.15) for tk in tickers]
allocation

In [ ]:
days = pd.date_range('2019-10-01', '2019-12-31', freq='BMS')

In [ ]:
days

Ponemos las fechas con formato: '%Y-%m-%d'

In [ ]:
days[0].strftime('%Y-%m-%d')

Mandamos las allocations:

In [ ]:
url = f'{url_base}/participants/allocation'
url_auth = f'{url}?key={user_key}'
print(url_auth)

for iday in days:
    str_date = iday.strftime('%Y-%m-%d')
    params = {
        'competi': competi,
        'algo_tag': algo_tag,
        'market': market,
        'date': str_date,
        'allocation': allocation
    }
    #print(json.dumps(params))
    response = requests.post(url_auth, data=json.dumps(params))
    print (response.json())

### Consultar las asignaciones de pesos registradas ###

In [ ]:
url = f'{url_base}/participants/algo_allocations'
params = {
    'key': user_key,
    'competi': competi,
    'algo_tag': algo_tag,
    'market': market,
}

In [ ]:
response = requests.get(url, params)

In [ ]:
def allocs_to_frame(json_allocations):
    alloc_list = []
    for json_alloc in json_allocations:
        #print(json_alloc)
        allocs = pd.DataFrame(json_alloc['allocations'])
        allocs.set_index('ticker', inplace=True)
        alloc_serie = allocs['alloc']
        alloc_serie.name = json_alloc['date'] 
        alloc_list.append(alloc_serie)
    all_alloc_df = pd.concat(alloc_list, axis=1).T
    return all_alloc_df

In [ ]:
allocs_to_frame(response.json())

### Verificar el algoritmo ###
Es una ejecución del algoritmo convirtiendo las asignaciones
en ordenes de compra y venta. Este paso no es necesario ejecutarlo
solo lo realiza el usuario en caso que quiera pre-visualizar su 
resultado

**Actualización API**: Debido a algunas restricciones de timeout de google cloud hemos actualizado este endpoint. El resultado saldrá inmediatamente si entra dentro del tiempo y sino se debe consultar con el endpoint *algo_exec_result*. Ver ejemplo más abajo

#### Ejecutar el backtesting

In [ ]:
url = f'{url_base}/participants/exec_algo'
url_auth = f'{url}?key={user_key}'
params = {
    'competi': competi,
    'algo_tag': algo_tag,
    'market': market,
    }

In [ ]:
response = requests.post(url_auth, data=json.dumps(params))

In [ ]:
if response.status_code == 200:
    exec_data = response.json()
    status = exec_data.get('status')
    print(status)
else:
    exec_data = dict()
    print(response.text)

La ejecución guarda una fecha de ejecución y un status. Cuando termina satisfactoriamente la clave 'content' tiene el resultado y los trades  

In [ ]:
res_data = exec_data.get('content')
trades = None
if res_data:
    print(pd.Series(res_data['result']))
    trades = pd.DataFrame(res_data['trades'])
trades

**Observacion**: Los algoritmos con muchas órdenes pueden no ejecutarse antes de 60 segundos, tiempo de timeout de las llamadas recibidas en google cloud.  En caso de recibir un error de timeout se puede obtener el resultado después de unos minutos, utilizando  el end_point a continuación

#### Ver el resultado del último backtesting

In [ ]:
url = f'{url_base}/participants/algo_exec_results'
params = {
    'key': user_key,
    'competi': competi,
    'algo_tag': algo_tag,
    'market': market,
}

Pasados unos 5 minutos

In [ ]:
response = requests.get(url, params)
exec_data = response.json()
print(exec_data.get('status'))
print(exec_data.get('content'))

### Re-iniciar las asginaciones para un algoritmo ###
En caso que se quiera reiniciar el algoritmo hay que 
eliminar todas las asignaciones incluyendo el mercado 
correspondiente

In [ ]:
url = f'{url_base}/participants/delete_allocations'
url_auth = f'{url}?key={user_key}'

params = {
    'competi': competi,
    'algo_tag': algo_tag,
    'market': market,
    }
response = requests.post(url_auth, data=json.dumps(params))

In [ ]:
response.text